In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
import keras
import tensorflow as tf
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator


CNN Model

In [2]:
# CNN Model
model = Sequential()

model.add(Conv2D(filters= 16 , kernel_size= (3,3), activation= 'relu'))

model.add(Conv2D(filters= 36 , kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 64 , kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 128 , kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate= 0.25))

model.add(Flatten())
model.add(Dense(units=64,activation= 'relu'))
model.add(Dropout(rate= 0.25))
model.add(Dense(units= 1, activation= 'sigmoid'))


# model.summary()


In [3]:
model.compile(optimizer='adam', loss= keras.losses.binary_crossentropy, metrics=['accuracy']) 

Preparing our Data using Data generator

In [4]:
def preprocessingpretrainedimages(path):
    # input : path 
    # output : pre trained images
    image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale= 1/225, horizontal_flip= True)  #data augmentation
    image = image_data.flow_from_directory(directory= path, target_size= (224,244),batch_size= 32, class_mode= 'binary')

    return image

def preprocessingpretestedimages(path):
    # input : path 
    # output : pre tested images
    image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale= 1/225, horizontal_flip= True) #data augmentation
    image = image_data.flow_from_directory(directory= path, target_size= (224,244),batch_size= 32, class_mode= 'binary')

    return image        

In [5]:
train_path = "Train"
test_path = "Test"

train_data = preprocessingpretrainedimages(train_path)
test_data = preprocessingpretrainedimages(test_path)

Found 1682 images belonging to 5 classes.
Found 299 images belonging to 5 classes.


Early Stopping & Model Checkpoints

In [6]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor="val_accuracy", min_delta= 1, patience= 3, verbose= 1, mode= 'auto')
model_checkpoints = ModelCheckpoint(monitor= "val_accuracy",filepath= "./bestmodel.h5",verbose= 1, save_best_only= True, mode = 'auto')


cd = [early_stopping,model_checkpoints]

Model Training

In [7]:
model_history = model.fit_generator(generator= train_data, steps_per_epoch= 8 , epochs= 30, verbose= 1, validation_data= test_data, validation_steps= 16,callbacks= cd )

C:\Users\hamod\AppData\Local\Temp\ipykernel_16572\142967850.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history = model.fit_generator(generator= train_data, steps_per_epoch= 8 , epochs= 30, verbose= 1, validation_data= test_data, validation_steps= 16,callbacks= cd )


: 

: 